In [1]:
# call relevant packages
from pyspark.sql import SparkSession
from pyspark.ml import Pipeline

from sparknlp.annotator import *
from sparknlp.common import *
from sparknlp.base import *

import sparknlp

# to use GPU 
spark = sparknlp.start()

print("Spark NLP version: ", sparknlp.version())
print("Apache Spark version: ", spark.version)

Spark NLP version:  3.1.0
Apache Spark version:  3.1.2


## Creating a composite CoNLL file 

### Create training set

In [2]:
# create the training files
with open("/Users/ramyabala/Research Projects/Evaluate NER/bio-splits/train/fold6_train.ner") as fp:
    text = fp.readlines()

In [3]:
text = "".join(text[1:]).split("\n\n") 

In [4]:
text[:10]

["'s\tVBZ\t(SQ*\tO\nhappening\tVBG\t(VP*\tO\nthere\tRB\t(ADVP*)\tO\ntonight\tNN\t(NP*)\tB-TIME\n,\t,\t*\tO\nGary\tNNP\t(NP*)))\tB-PERSON\n?\t.\t*))\tO",
 'What\tWP\t(TOP(SBARQ(WHNP(WHNP*)\tO\nexactly\tRB\t(ADVP*))\tO\nis\tVBZ\t(SQ*\tO\nthe\tDT\t(NP*\tO\ntruth\tNN\t*))\tO\n?\t.\t*))\tO',
 'His\tPRP$\t(TOP(S(NP*\tO\nplatform\tNN\t*)\tO\nincludes\tVBZ\t(VP*\tO\nsubsidies\tNNS\t(NP(NP*)\tO\nto\tTO\t(SBAR(S(VP*\tO\nassist\tVB\t(VP*\tO\nthe\tDT\t(NP(NP(NP*\tO\nchildren\tNNS\t*)\tO\nof\tIN\t(PP*\tO\nlow\tJJ\t(NP(NML*\tO\n-\tHYPH\t*\tO\nincome\tNN\t*)\tO\nfamilies\tNNS\t*)))\tO\nwho\tWP\t(SBAR(WHNP*)\tO\nattend\tVBP\t(S(VP*\tO\nprivate\tJJ\t(NP*\tO\nsecondary\tJJ\t*\tO\nschools\tNNS\t*)))))))))))\tO\n.\t.\t*))\tO',
 "One\tCD\t(TOP(FRAG(NP*\tB-CARDINAL\nexample\tNN\t*)\tO\nhe\tPRP\t(SBAR(S(NP*)\tO\ngives\tVBZ\t(VP*)))\tO\n:\t:\t*\tO\n``\t``\t*\tO\nShe\tPRP\t(S(NP*)\tO\ndid\tVBD\t(VP*\tO\nn't\tRB\t*\tO\nask\tVB\t(VP*\tO\n''\t''\t*\tO\n-LRB-\t-LRB-\t*\tO\nwhy\tWRB\t(SBAR(WHADVP*)\tO\nthe\tDT\t(S(

In [5]:
import pandas as pd
df = pd.DataFrame([x.split('\t') for x in text[1].split('\n')], columns=["Token","Pos","Pos_special","Entity_label"])

In [6]:
# creating the training data
conll_lines = "-DOCSTART- -X- -X- -O-\n\n"
for t in range(len(text)):    
    df = pd.DataFrame([x.split('\t') for x in text[t].split('\n') if len(x.split('\t')) == 4], columns=["Token","Pos","Pos_special","Entity_label"])
    tokens = df.Token.tolist()
    pos_labels = df.Pos.tolist()
    entity_labels = df.Entity_label.tolist()
    for token, pos, label in zip(tokens,pos_labels,entity_labels):
        conll_lines += "{} {} {} {}\n".format(token, pos, pos, label)
    conll_lines += "\n"

In [7]:
with open("/Users/ramyabala/Research Projects/Evaluate NER/bio-splits/train/sample.train","w") as fp:
    for line in conll_lines:
        fp.write(line)

### Create dev set

In [8]:
# create the training filJe with all the news corpus
with open("/Users/ramyabala/Research Projects/Evaluate NER/bio-splits/dev/fold6_dev.ner") as fp:
    text = fp.readlines()

In [9]:
text = "".join(text[1:]).split("\n\n") 

In [10]:
text[:10]

['leaving\tVBG\t(S(VP*\tO\nJuilliard\tNNP\t(NP*))))\tB-ORG\n,\t,\t*\tO\nMr.\tNNP\t(NP*\tO\nMcDuffie\tNNP\t*)\tB-PERSON\nhas\tVBZ\t(VP*\tO\nmade\tVBN\t(VP*\tO\nsome\tDT\t(NP(NP*\tO\nsmart\tJJ\t*\tO\nmoves\tNNS\t*)\tO\nand\tCC\t*\tO\nsome\tDT\t(NP*\tO\ncontroversial\tJJ\t*\tO\nones\tNNS\t*))))\tO\n.\t.\t*))\tO',
 "They\tPRP\t(TOP(S(NP*)\tO\n're\tVBP\t(VP*\tO\nsuing\tVBG\t(VP*\tO\n27\tCD\t(NP(NP*\tB-CARDINAL\ndefendants\tNNS\t*)\tO\nincluding\tVBG\t(PP*\tO\nthe\tDT\t(NP(NP*\tO\nclub\tNN\t*)\tO\n,\t,\t*\tO\nthe\tDT\t(NP*\tO\nband\tNN\t*)\tO\n,\t,\t*\tO\ncompanies\tNNS\t(NP(NP*)\tO\nwith\tIN\t(PP*\tO\nties\tNNS\t(NP(NP*)\tO\nto\tIN\t(PP*\tO\nthe\tDT\t(NP*\tO\nclub\tNN\t*)))))\tO\n,\t,\t*\tO\nthe\tDT\t(NP(NP*\tO\ntown\tNN\t*)\tO\nas\tRB\t(CONJP*\tO\nwell\tRB\t*\tO\nas\tIN\t*)\tO\nthe\tDT\t(NP*\tO\nstate\tNN\t*)))))))\tO\n.\t.\t*))\tO",
 'Well\tUH\t(TOP(S(INTJ*)\tO\n,\t,\t*\tO\nfrom\tIN\t(PP*\tO\nthis\tDT\t(NP*\tO\nfootage\tNN\t*))\tO\n,\t,\t*\tO\nwe\tPRP\t(NP*)\tO\ncan\tMD\t(VP*\tO\nsee\tVB\

In [11]:
import pandas as pd
df = pd.DataFrame([x.split('\t') for x in text[1].split('\n')], columns=["Token","Pos","Pos_special","Entity_label"])

In [12]:
# creating the training data
conll_lines = "-DOCSTART- -X- -X- -O-\n\n"
for t in range(len(text)):    
    df = pd.DataFrame([x.split('\t') for x in text[t].split('\n') if len(x.split('\t')) == 4], columns=["Token","Pos","Pos_special","Entity_label"])
    tokens = df.Token.tolist()
    pos_labels = df.Pos.tolist()
    entity_labels = df.Entity_label.tolist()
    for token, pos, label in zip(tokens,pos_labels,entity_labels):
        conll_lines += "{} {} {} {}\n".format(token, pos, pos, label)
    conll_lines += "\n"

In [13]:
with open("/Users/ramyabala/Research Projects/Evaluate NER/bio-splits/dev/sample.train","w") as fp:
    for line in conll_lines:
        fp.write(line)

### Create test set

In [14]:
# create the training filJe with all the news corpus
with open("/Users/ramyabala/Research Projects/Evaluate NER/bio-splits/test/fold6_test.ner") as fp:
    text = fp.readlines()

In [15]:
text = "".join(text[1:]).split("\n\n") 

In [16]:
text[:10]

['subway\tNN\t*)\tO\nto\tIN\t(PP*\tO\nDisney\tNNP\t(NP*)))\tB-FAC\nhas\tVBZ\t(VP*\tO\nalready\tRB\t(ADVP*)\tO\nbeen\tVBN\t(VP*\tO\nconstructed\tVBN\t(VP*)))\tO\n.\t.\t*))\tO',
 'Ah\tUH\t(TOP(S(INTJ*)\tO\n,\t,\t*\tO\neverything\tNN\t(NP*)\tO\nis\tVBZ\t(VP*\tO\ntop\tJJ\t(ADJP*\tO\nsecret\tJJ\t*))\tO\n.\t.\t*))\tO',
 'The\tDT\t(TOP(S(NP(NP*\tO\narea\tNN\t*)\tO\nof\tIN\t(PP*\tO\nHong\tNNP\t(NP*\tB-GPE\nKong\tNNP\t*)))\tI-GPE\nis\tVBZ\t(VP*\tO\nonly\tRB\t(NP(QP*\tO\none\tCD\t*\tB-QUANTITY\nthousand\tCD\t*\tI-QUANTITY\n-\tHYPH\t*\tI-QUANTITY\nplus\tCC\t*)\tI-QUANTITY\nsquare\tJJ\t*\tI-QUANTITY\nkilometers\tNNS\t*))\tI-QUANTITY\n.\t.\t*))\tO',
 "Here\tRB\t(TOP(S(ADVP*)\tO\n,\t,\t*\tO\ntourists\tNNS\t(NP*)\tO\ncan\tMD\t(VP(VP*\tO\nlearn\tVB\t(VP*\tO\nabout\tIN\t(PP*\tO\nHong\tNNP\t(NP(NP*\tB-GPE\nKong\tNNP\t*\tI-GPE\n's\tPOS\t*)\tI-GPE\nfilm\tNN\t*\tO\nhistory\tNN\t*))))\tO\n,\t,\t*\tO\nand\tCC\t*\tO\ncan\tMD\t(VP*\tO\ncome\tVB\t(VP*\tO\nup\tRB\t(ADVP*\tO\nclose\tRB\t*)\tO\nwith\tIN\t(PP*\tO\n

In [17]:
import pandas as pd
df = pd.DataFrame([x.split('\t') for x in text[1].split('\n')], columns=["Token","Pos","Pos_special","Entity_label"])

In [18]:
# creating the training data
conll_lines = "-DOCSTART- -X- -X- -O-\n\n"
for t in range(len(text)):    
    df = pd.DataFrame([x.split('\t') for x in text[t].split('\n') if len(x.split('\t')) == 4], columns=["Token","Pos","Pos_special","Entity_label"])
    tokens = df.Token.tolist()
    pos_labels = df.Pos.tolist()
    entity_labels = df.Entity_label.tolist()
    for token, pos, label in zip(tokens,pos_labels,entity_labels):
        conll_lines += "{} {} {} {}\n".format(token, pos, pos, label)
    conll_lines += "\n"

In [19]:
with open("/Users/ramyabala/Research Projects/Evaluate NER/bio-splits/test/sample.train","w") as fp:
    for line in conll_lines:
        fp.write(line)

### import data in CONLL format

In [20]:
# importing the training set
from sparknlp.training import CoNLL

training_data = CoNLL().readDataset(spark, '/Users/ramyabala/Research Projects/Evaluate NER/bio-splits/train/sample.train')
training_data.show()

+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|                text|            document|            sentence|               token|                 pos|               label|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|'s happening ther...|[{document, 0, 34...|[{document, 0, 34...|[{token, 0, 1, 's...|[{pos, 0, 1, VBZ,...|[{named_entity, 0...|
|What exactly is t...|[{document, 0, 26...|[{document, 0, 26...|[{token, 0, 3, Wh...|[{pos, 0, 3, WP, ...|[{named_entity, 0...|
|His platform incl...|[{document, 0, 11...|[{document, 0, 11...|[{token, 0, 2, Hi...|[{pos, 0, 2, PRP$...|[{named_entity, 0...|
|One example he gi...|[{document, 0, 11...|[{document, 0, 11...|[{token, 0, 2, On...|[{pos, 0, 2, CD, ...|[{named_entity, 0...|
|and then horrendo...|[{document, 0, 52...|[{document, 0, 52...|[{token, 0, 2, an...|[{pos, 0, 2, CC, ..

In [21]:
dev_data = CoNLL().readDataset(spark, '/Users/ramyabala/Research Projects/Evaluate NER/bio-splits/dev/sample.train')
dev_data.show()

+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|                text|            document|            sentence|               token|                 pos|               label|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|leaving Juilliard...|[{document, 0, 87...|[{document, 0, 87...|[{token, 0, 6, le...|[{pos, 0, 6, VBG,...|[{named_entity, 0...|
|They 're suing 27...|[{document, 0, 12...|[{document, 0, 12...|[{token, 0, 3, Th...|[{pos, 0, 3, PRP,...|[{named_entity, 0...|
|Well , from this ...|[{document, 0, 27...|[{document, 0, 27...|[{token, 0, 3, We...|[{pos, 0, 3, UH, ...|[{named_entity, 0...|
|Scientists have s...|[{document, 0, 25...|[{document, 0, 25...|[{token, 0, 9, Sc...|[{pos, 0, 9, NNS,...|[{named_entity, 0...|
|Had they come to ...|[{document, 0, 11...|[{document, 0, 11...|[{token, 0, 2, Ha...|[{pos, 0, 2, VBD,..

In [22]:
test_data = CoNLL().readDataset(spark, '/Users/ramyabala/Research Projects/Evaluate NER/bio-splits/test/sample.train')
test_data.show()

+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|                text|            document|            sentence|               token|                 pos|               label|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|subway to Disney ...|[{document, 0, 46...|[{document, 0, 46...|[{token, 0, 5, su...|[{pos, 0, 5, NN, ...|[{named_entity, 0...|
|Ah , everything i...|[{document, 0, 30...|[{document, 0, 30...|[{token, 0, 1, Ah...|[{pos, 0, 1, UH, ...|[{named_entity, 0...|
|The area of Hong ...|[{document, 0, 68...|[{document, 0, 68...|[{token, 0, 2, Th...|[{pos, 0, 2, DT, ...|[{named_entity, 0...|
|Here , tourists c...|[{document, 0, 11...|[{document, 0, 11...|[{token, 0, 3, He...|[{pos, 0, 3, RB, ...|[{named_entity, 0...|
|The answer is tha...|[{document, 0, 97...|[{document, 0, 97...|[{token, 0, 2, Th...|[{pos, 0, 2, DT, ..

### Training the model - 10 epochs

In [23]:
# use bert embeddings
bert = BertEmbeddings.pretrained('bert_base_cased', 'en').setInputCols(["sentence",'token']).setOutputCol("bert").setCaseSensitive(True)#.setMaxSentenceLength(512)

bert_base_cased download started this may take some time.
Approximate size to download 389.1 MB
[OK!]


In [24]:
# looking at the shape - using count as its not very large dataset
training_data.count()

62138

In [25]:
# transforming the training data into embeddings and saving it as parquet files
readyTrainingData = bert.transform(training_data)

readyTrainingData.write.mode("Overwrite").parquet("/tmp/conll2003/bert_train")

In [26]:
readyTrainingData = spark.read.parquet("/tmp/conll2003/bert_train")

In [27]:
# transforming the development data into embeddings and saving it as parquet files
readyDevData = bert.transform(dev_data)

readyDevData.write.mode("Overwrite").parquet("/tmp/conll2003/bert_dev")

In [28]:
readyDevData = spark.read.parquet("/tmp/conll2003/bert_dev")

In [29]:
# transforming the test data into embeddings and saving it as parquet files
readyTestData = bert.transform(test_data)

readyTestData.write.mode("Overwrite").parquet("/tmp/conll2003/bert_test")

In [30]:
readyTestData = spark.read.parquet("/tmp/conll2003/bert_test")

### Dev set - Fold 2 dataset

In [31]:
# initialize NER tagger
nerTagger = NerDLApproach()\
.setInputCols(["sentence", "token", "bert"])\
.setLabelColumn("label")\
.setOutputCol("ner")\
.setMaxEpochs(10)\
.setBatchSize(4)\
.setEnableMemoryOptimizer(True)\
.setRandomSeed(0)\
.setVerbose(1)\
.setValidationSplit(0.2)\
.setEvaluationLogExtended(True)\
.setEnableOutputLogs(True)\
.setIncludeConfidence(True)\
.setTestDataset("/tmp/conll2003/bert_dev")

In [ ]:
# train the model
%time myNerModel = nerTagger.fit(readyTrainingData)

### Testing on Fold 1 Dataset

In [33]:
# infer from the trained model
%time results = myNerModel.transform(readyTestData).select("sentence","token","label","ner").collect()

CPU times: user 3.94 s, sys: 658 ms, total: 4.6 s
Wall time: 32 s


In [34]:
# to find exceptions where no. of labels does not match no. of ners detected
count = 0
indices = []
for i,row in enumerate(results):
    if len(row['label']) != len(row['ner']):
        count += 1
        indices.append(i)

In [35]:
print(count)
print(indices)

6
[2465, 2673, 3216, 3742, 3842, 7121]


In [36]:
exclusion_list = [results[t] for t in indices]
results = [results[i] for i in range(len(results)) if i not in indices]

In [37]:
tokens = []
labels = []
ners = []

for row in results:
    tokens.append([t['result'] for t in row['token']])
    labels.append([t['result'] for t in row['label']])
    ners.append([t['result'] for t in row['ner']])

In [38]:
from seqeval.metrics import accuracy_score, f1_score, classification_report
print(accuracy_score(labels,ners))

0.9816502638710598


In [39]:
print(f1_score(labels,ners))

0.9006251749556778


In [40]:
print(classification_report(labels,ners, zero_division=1))

              precision    recall  f1-score   support

    CARDINAL       0.84      0.88      0.86       934
        DATE       0.86      0.90      0.88      1367
       EVENT       0.76      0.68      0.72       103
         FAC       0.61      0.76      0.68       103
         GPE       0.95      0.96      0.95      2031
    LANGUAGE       0.93      0.89      0.91        45
         LAW       0.65      0.71      0.68        42
         LOC       0.88      0.70      0.78       193
       MONEY       0.91      0.92      0.91       304
        NORP       0.96      0.93      0.95       892
     ORDINAL       0.85      0.91      0.88       204
         ORG       0.88      0.89      0.89      1690
     PERCENT       0.92      0.92      0.92       231
      PERSON       0.93      0.96      0.94      2027
     PRODUCT       0.76      0.76      0.76        67
    QUANTITY       0.84      0.78      0.81        86
        TIME       0.65      0.70      0.67       169
 WORK_OF_ART       0.76    